In [3]:
import cobra
import pandas

import math
nlog2 = math.log(2)

def doubling_time(model):
    with model as m:
        ov = m.slim_optimize()
        if ov < 1e-6:
            growth = 'No growth'
        else:  
            growth = nlog2*60/ov
            growth = str(round(growth, 3))
        
    return growth

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [4]:
#Define Medias
gcb_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_h2o_e_','EX_co2_e_','EX_h_e_','EX_cys_L_e_','EX_mn2_e_','EX_mobd_e_','EX_cobalt2_e_','EX_cu2_e_','EX_zn2_e_','EX_pi_e_','EX_na1_e_','EX_cl_e_','EX_ala_L_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_so4_e_','EX_ca2_e_','EX_fe3_e_','EX_mg2_e_','EX_k_e_','EX_gln_L_e_'])
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])
mdm_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_co2_e_', 'EX_cobalt2_e_', 'EX_cu2_e_' , 'EX_h_e_', 'EX_h2o_e_','EX_mn2_e_','EX_mobd_e_','EX_tungs_e_', 'EX_zn2_e_','EX_na1_e_','EX_cl_e_','EX_k_e_','EX_so4_e_','EX_nh4_e_','EX_mg2_e_','EX_pi_e_','EX_ca2_e_','EX_fe3_e_','EX_no3_e_','EX_asp_L_e_','EX_glu_L_e_','EX_arg_L_e_','EX_gly_e_','EX_ser_L_e_','EX_leu_L_e_','EX_ile_L_e_','EX_val_L_e_','EX_tyr_L_e_','EX_cys_L_e_','EX_pro_L_e_','EX_trp_L_e_','EX_thr_L_e_','EX_phe_L_e_','EX_asn_L_e_','EX_gln_L_e_','EX_his_L_e_','EX_met_L_e_','EX_ala_L_e_','EX_lys_L_e_','EX_gthrd_e_','EX_thm_e_','EX_pnto_R_e_','EX_glc_D_e_','EX_hxan_e_','EX_ura_e_'])

def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000. 
            
def GCB(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in gcb_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000. 
            
def MDM(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000. 
            
def MDM_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -27.8
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.48
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -3.76
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.17
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.84
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.33
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -1.12
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.51
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.69
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.34
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.43
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -107.94697
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -11.48
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -5.74
        if 'EX_nh4_e_' in reaction.id:
            reaction.lower_bound = -4.11
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -1.07
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.27
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -40
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.254
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.051
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -0.71
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.23
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.65
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.39
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.42
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.13
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.15
        if 'EX_thm_e_' in reaction.id:
            reaction.lower_bound = -0.00697
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.004
        if 'EX_hxan_e_' in reaction.id:
            reaction.lower_bound = -0.37
        if 'EX_ura_e_' in reaction.id:
            reaction.lower_bound = -0.45  
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -105.018
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000. 
        elif reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20
            
def GCB_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in gcb_req:
            reaction.lower_bound = -1000.
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -22.20299
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -2.28370028
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -5.747441637
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.3406048688
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -8.156091275
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -12.98846624
        if 'EX_ala_L_e_' in reaction.id:
            reaction.lower_bound = -8.754876915
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -44.81575602
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -6.403761753
        if 'EX_gln_L_e_' in reaction.id:
            reaction.lower_bound = -0.68425
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -4.95090354
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -3.1781758
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -4.309531041
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -28.01410255
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -32.69853966
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -127.3607438
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -113.2400004
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.04940366286
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -0.06356716725
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -0.57774826
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -3.702619388
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -0.3745310559
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -1.256810949
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -3.65929243
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -1.306871531
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -1.888864264
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.2203449917
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -1.324587377
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.00371
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000. 
        elif reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
            
def rpmi_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -1000.    
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -11.1012
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.2855
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -0.1503
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.3784
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -0.1359
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.1332
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.1707
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.1737
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.0908
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.424
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.847
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -5.3655
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -108.6993
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -137.9818
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -6.354
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -1.1481
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -10.416
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.0967
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1005
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1679
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.0245
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1104
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.001
        if 'EX_chol_e_' in reaction.id:
            reaction.lower_bound = -0.0215
        if 'EX_inost_e_' in reaction.id:
            reaction.lower_bound = -0.1943
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.0033
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.         
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.

def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-1000.   



In [6]:
model=read_model("C://Users/Aimee/Documents/Metabolic_Modeling/Gc_GENRE_2022/Models/NGO_557.json")
model.objective= 'biomass'

In [7]:

print (model.id)
MDM(model)
print ('MDM:',model.slim_optimize())
print ('MDM:',doubling_time(model))
MDM_molarity(model)
print ('MDM_molarity:',model.slim_optimize())
print ('MDM_molarity:',doubling_time(model))
rpmi(model)
print ('RPMI:', model.slim_optimize())
print ('RPMI:',doubling_time(model))
rpmi_molarity(model)
print ('RPMI_Molarity:', model.slim_optimize())
print ('RPMI_Molarity:',doubling_time(model))
GCB(model)
print ('GCB:',model.slim_optimize())
print ('GCB:',doubling_time(model))
GCB_molarity(model)
print ('GCB_molarity:',model.slim_optimize())
print ('GCB_Molartiy:',doubling_time(model))
complete(model)
print ('Complete media:', model.slim_optimize())
print ('Complete media:',doubling_time(model))


iNGO_557
MDM: 1.4529034642314587
MDM: 28.625
MDM_molarity: 1.5455891894179035
MDM_molarity: 26.908
RPMI: 1.3818431667483044
RPMI: 30.097
RPMI_Molarity: 0.2849964780786306
RPMI_Molarity: 145.928
GCB: 1.4025538875408128
GCB: 29.652
GCB_molarity: 1.4732076073932825
GCB_Molartiy: 28.23
Complete media: 33.81006161676428
Complete media: 1.23


In [7]:
def rpmi_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.   
            
def GCB_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in gcb_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.     

def MDM_with_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.   
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.   
            
def rpmi_only_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.
            
def GCB_only_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in gcb_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.  
            
def MDM_only_lactate(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10. 
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.    

def MDM_no_carbon(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0.   
        if reaction.id == 'EX_h_e_':
            reaction.lower_bound = -1000.   


In [8]:
#When media metabolites are provided in infinite amounts, growth on carbon sources is unrealisitic 
#and lactate is almost never used as a carbon source. 
#This seems to be due to the use of serine and alanine as sources of pyruvate. Elimination of these reactions restores growth on lactate alone. 
#We know that Gc uses lactate as a carbon source in these medias based on experimental data.


In [9]:
with model:
    MDM_no_carbon(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

43.863 minutes
Lactate Flux 0.0
PYK Flux 0.0


In [16]:
with model:  
    MDM(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

28.625 minutes
Lactate Flux 0.0
PYK Flux 8.658085307041807


In [11]:
with model:  
    MDM_with_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

28.625 minutes
Lactate Flux 0.0
PYK Flux 8.65808530703487


In [12]:
with model:   
    MDM_only_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

38.738 minutes
Lactate Flux -10.0
PYK Flux 0.0


In [14]:
with model:   
    MDM(model)
    for reaction in model.reactions:
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_pyr_e_':
            reaction.lower_bound = -10.
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

36.968 minutes
Lactate Flux 0.0
PYK Flux 0.0


In [27]:
with model:   
    MDM_molarity(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))
    
with model:   
    MDM_molarity(model)
    for reaction in model.reactions:
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

with model:   
    MDM_molarity(model)
    for reaction in model.reactions:
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -54 
        if reaction.id == 'EX_pyr_e_':
            reaction.lower_bound = -0.
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

with model:   
    MDM_molarity(model)
    for reaction in model.reactions:
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -0.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -0. 
        if reaction.id == 'EX_pyr_e_':
            reaction.lower_bound = -54
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print('PYK Flux ' + str(model.reactions.PYK.flux))

26.908 minutes
Lactate Flux 0.0
PYK Flux 25.456093540162612
247.18 minutes
Lactate Flux 0.0
PYK Flux 0.0
43.191 minutes
Lactate Flux -24.34692128910867
PYK Flux 0.0
28.207 minutes
Lactate Flux 0.0
PYK Flux 0.0


In [93]:
with model:
    GCB(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))

29.652 minutes
Lactate Flux 0.0


In [94]:
with model:
    GCB_with_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))

29.652 minutes
Lactate Flux 0.0


In [95]:
with model:
    GCB_only_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    print(model.summary())

39.75 minutes
Lactate Flux -10.0
Objective
1.0 biomass = 1.0462473704951127

Uptake
------
Metabolite      Reaction     Flux  C-Number C-Flux
   ala_L_e   EX_ala_L_e_   0.9287         3  2.53%
   arg_L_e   EX_arg_L_e_   0.3413         6  1.86%
   asn_L_e   EX_asn_L_e_   0.2632         4  0.96%
     ca2_e     EX_ca2_e_ 0.004956         0  0.00%
      cl_e      EX_cl_e_ 0.004956         0  0.00%
 cobalt2_e EX_cobalt2_e_ 0.003304         0  0.00%
     cu2_e     EX_cu2_e_ 0.003304         0  0.00%
   cys_L_e   EX_cys_L_e_  0.06433         3  0.18%
     fe3_e     EX_fe3_e_  0.01534         0  0.00%
   gln_L_e   EX_gln_L_e_    1.143         5  5.19%
     gly_e     EX_gly_e_       10         2 18.17%
       h_e       EX_h_e_    10.88         0  0.00%
   his_L_e   EX_his_L_e_   0.1058         6  0.58%
   ile_L_e   EX_ile_L_e_   0.3054         6  1.66%
       k_e       EX_k_e_   0.1858         0  0.00%
   lac_L_e   EX_lac_L_e_       10         3 27.25%
   leu_L_e   EX_leu_L_e_   0.4945         

In [96]:
with model:
    rpmi_with_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))

30.097 minutes
Lactate Flux 0.0


In [97]:
with model:
    rpmi_only_lactate(model)
    print(doubling_time(model)+ ' minutes')
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))

40.883 minutes
Lactate Flux -10.0


In [98]:
#However, the media does give more appropriate growth predictions if using the molarity specific media formulations


In [104]:
#Growth is on no carbon source is fueled by alanine/serine/glycine. 
#in the absence of these enzymes, lactate use is oxygen dependent. 
with model:
    rpmi(model)
   
    model.reactions.SERD_L.knock_out()
    model.reactions.ALATA_L.knock_out()
    model.reactions.EX_glc_D_e_.lower_bound = 0
    model.reactions.EX_lac_L_e_.lower_bound = 0
    model.reactions.EX_o2_e_.lower_bound = -20
    print('growth rate on no carbon source ' + doubling_time(model))
    
    model.reactions.EX_glc_D_e_.lower_bound = -11
    model.reactions.EX_lac_L_e_.lower_bound = 0
    model.reactions.EX_o2_e_.lower_bound = -20
    print('growth rate on GCB with glucose only ' + doubling_time(model))
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
        
    model.reactions.EX_glc_D_e_.lower_bound = 0
    model.reactions.EX_lac_L_e_.lower_bound = -22
    model.reactions.EX_o2_e_.lower_bound = -20
    print('growth rate on GCB with lactate only ' + doubling_time(model))
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
        
    model.reactions.EX_glc_D_e_.lower_bound = -11
    model.reactions.EX_lac_L_e_.lower_bound = -22
    model.reactions.EX_o2_e_.lower_bound = -20
    print('growth rate on GCB with both glucose and lactate ' + doubling_time(model))
    print('Lactate Flux ' + str(model.reactions.EX_lac_L_e_.flux))
    

growth rate on no carbon source No growth
growth rate on GCB with glucose only 33.23
Lactate Flux 0.0
growth rate on GCB with lactate only 45.267
Lactate Flux -22.0
growth rate on GCB with both glucose and lactate 32.259
Lactate Flux -4.0130912050579255


In [106]:
#Test if addition of an amino acid will improve the growth of GC in RPMI using the "equally scaled" definiation of RPMI
with model:
    rpmi(model)
    print("Original doubling time " + doubling_time(model))
    original_growth_rate = round(model.slim_optimize(),3)
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            if reaction.lower_bound >=0:
                pass
            if reaction.lower_bound < 0:
                reaction.lower_bound = -50.
                new_growth_rate=round(model.slim_optimize(),3)
                if new_growth_rate > original_growth_rate:
                    print(reaction.id +":     " + doubling_time(model))
            rpmi(model)


Original doubling time 30.097
EX_glc_D_e_:     22.542
EX_o2_e_:     16.668
EX_ser_L_e_:     24.6


In [108]:
#Test if addition of an amino acid will improve the growth of GC in RPMI using the "molarity" definiation of RPMI

with model:
    rpmi_molarity(model)
    print("Original doubling time " + doubling_time(model))
    original_growth_rate = round(model.slim_optimize(),3)
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            if reaction.lower_bound >=0:
                pass
            if reaction.lower_bound < 0:
                reaction.lower_bound=(reaction.lower_bound)*5
                new_growth_rate=round(model.slim_optimize(),3)
                if new_growth_rate > original_growth_rate:
                    print(reaction.id+":       "+doubling_time(model))
            rpmi_molarity(model)


Original doubling time 145.928
EX_asn_L_e_:       55.955
EX_asp_L_e_:       106.94
EX_glu_L_e_:       109.688
EX_gly_e_:       110.222
EX_pro_L_e_:       103.015
EX_ser_L_e_:       87.747


In [46]:
#Send results to CSV
#Define media components found in RPMI to list. 
with model:
    rpmi_molarity(model)
    aminoacidsupplementation=[]
    aminoacidsupplementation.append('original RPMI')
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            if reaction.lower_bound >=0:
                pass
            if reaction.lower_bound < 0:
                aminoacidsupplementation.append(reaction.id)

#create a dataframe to store the simulation result on list of carbon source
growthCapabilities=pandas.DataFrame(index=aminoacidsupplementation)
growthCapabilities['NGO_557'] = ''

#determine new growth rate if each media component is supplemented
listCapabilities=[]

with model: 
    rpmi_molarity(model)
    listCapabilities.append(doubling_time(model))
    for source in aminoacidsupplementation:
            if source not in model.reactions:
                pass
            if source in model.reactions:
                x= model.reactions.get_by_id(source).lower_bound
                model.reactions.get_by_id(source).lower_bound=x*5
                listCapabilities.append(doubling_time(model))
                rpmi_molarity(model)

    for col in growthCapabilities.columns:
        growthCapabilities[col]=listCapabilities

growthCapabilities.to_csv("growth_on_rpmi_with_supplementation.csv")
growthCapabilities


,NGO_557
original RPMI,145.928
EX_arg_L_e_,145.928
EX_asn_L_e_,55.955
EX_asp_L_e_,106.94
EX_ca2_e_,145.928
EX_chol_e_,145.928
EX_cl_e_,145.928
EX_co2_e_,145.928
EX_cobalt2_e_,145.928
EX_cu2_e_,145.928


In [15]:
#Test if addition of an amino acid will improve the growth of GC in RPMI using the "equally scaled" definiation of RPMI

with model:
    rpmi(model)
    print("Original biomass " + str(model.slim_optimize()))
    original_growth_rate = round(model.slim_optimize(),3)
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            if reaction.lower_bound >=0:
                pass
            if reaction.lower_bound < 0:
                reaction.lower_bound = -50.
                new_growth_rate=round(model.slim_optimize(),3)
                if new_growth_rate > original_growth_rate:
                    print(reaction.id +":     " + str(model.slim_optimize()))
            rpmi(model)


Original biomass 1.3818431668560585
EX_glc_D_e_:     1.8449790733782798
EX_o2_e_:     2.4951676818197828
EX_ser_L_e_:     1.6906004378309958


In [16]:
#Test if addition of an amino acid will improve the growth of GC in RPMI using the "molarity" definiation of RPMI

with model:
    rpmi_molarity(model)
    print("Original biomass " + str(model.slim_optimize()))
    original_growth_rate = round(model.slim_optimize(),3)
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            if reaction.lower_bound >=0:
                pass
            if reaction.lower_bound < 0:
                reaction.lower_bound=(reaction.lower_bound)*5
                new_growth_rate=round(model.slim_optimize(),3)
                if new_growth_rate > original_growth_rate:
                    print(reaction.id+":       "+ str(model.slim_optimize()))
            rpmi_molarity(model)


Original biomass 0.2849964780784725
EX_asn_L_e_:       0.7432505946020238
EX_asp_L_e_:       0.38889759856186784
EX_glu_L_e_:       0.37915577951248697
EX_gly_e_:       0.37731786101215536
EX_pro_L_e_:       0.40371696721611483
EX_ser_L_e_:       0.4739617236586696
